## Install/load libraries

Notebook utilises tcrdist to generate a tcr specific distance matrix for each chain of the TCRs input into this. See the link below for full documentation.

https://tcrdist3.readthedocs.io/en/latest/tcrdistances.html

To use this notebook:
1. Upload the files from outputs/{CHAIN}/*.csv to the session (can drag and drop from Finder.
2. Create a folder to hold the output, using in a new code block ```! mkdir tcrdist```
3. Run the install chunk, then the chunk for either alpha or beta chain
4. Download the zipped results file (semi-automated, will need to allow on the first go in the session).
5. If running both chains, remove all files (except the pre-loaded 'sample_data' folder), including tcrdist results folder, and run steps 3-4 again.

In [ ]:
!pip install tcrdist3

import pandas as pd
from tcrdist.pgen import OlgaModel
from tcrdist import mappers
from tcrdist.repertoire import TCRrep
from tcrdist.public import TCRpublic
from tcrdist.setup_tests import download_and_extract_zip_file

import re
import os

from google.colab import files

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 5.6 MB/s eta 0:00:00
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2434 sha256=3a5b30db430c851a06593a38a69ead4e685c3e9cf2843885990f7183f4482383
  Stored in directory: /root/.cache/pip/wheels/dd/02/6b/2f0f7e1c0454094c880e63076da142e67f1ba72f82baa6195f
Successfully built feather-format


## Run beta chains through this code block

In [ ]:
file_ = [i for i in os.listdir() if re.search('.csv', i)]
for i in file_:
  df = pd.read_csv(i)

  df = df.rename(columns = {'v_call': 'v_b_gene',
                            'j_call': 'j_b_gene',
                            'junction_aa': 'cdr3_b_aa'})

  df['v_b_gene'] = [f'{i}*01' for i in df['v_b_gene']] # append fake allele for TCRdist, doesn't recognise decomb ids without alleles
  df['j_b_gene'] = [f'{i}*01' for i in df['j_b_gene']] # append fake allele for TCRdist, doesn't recognise decomb ids without alleles

  tr = TCRrep(cell_df = df,
              organism = 'mouse',
              chains = ['beta'],
              db_file = 'alphabeta_gammadelta_db.tsv',
              store_all_cdr = False)

  olga_beta  = OlgaModel(chain_folder = "mouse_T_beta", recomb_type="VDJ")

  tr.clone_df['pgen_cdr3_b_aa'] = olga_beta.compute_aa_cdr3_pgens(CDR3_seq = tr.clone_df.cdr3_b_aa)

  tr.clone_df.to_csv(f"tcrdist/{re.sub('.csv', '', i)}_pgens.csv", index = False)

  tmp = pd.DataFrame(tr.pw_beta)
  tmp.columns = tr.clone_df.id_nt.to_list()

  tmp.to_csv(f"tcrdist/{re.sub('.csv', '', i)}_tcrdistmatrix.csv", index = True, compression = 'gzip')

!tar -czvf tcrdist.tar.gz tcrdist
files.download("/content/tcrdist.tar.gz")

/usr/local/lib/python3.12/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/python3.12/dist-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")


tcrdist/
tcrdist/.ipynb_checkpoints/
tcrdist/0_complete-data_tcrdistmatrix.csv
tcrdist/0_complete-data_pgens.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Run alpha chains through this code block

In [ ]:
file_ = [i for i in os.listdir() if re.search('.csv', i)]
for i in file_:
  df = pd.read_csv(i)

  df = df.rename(columns = {'v_call': 'v_a_gene',
                            'j_call': 'j_a_gene',
                            'junction_aa': 'cdr3_a_aa'})

  df['v_a_gene'] = [f'{i}*01' for i in df['v_a_gene']] # append fake allele for TCRdist, doesn't recognise decomb id without alleles
  df['j_a_gene'] = [f'{i}*01' for i in df['j_a_gene']] # append fake allele for TCRdist, doesn't recognise decomb id without alleles

  tr = TCRrep(cell_df = df,
              organism = 'mouse',
              chains = ['alpha'],
              db_file = 'alphabeta_gammadelta_db.tsv',
              store_all_cdr = False)

  olga_beta  = OlgaModel(chain_folder = "mouse_T_alpha", recomb_type="VJ")

  tr.clone_df['pgen_cdr3_a_aa'] = olga_beta.compute_aa_cdr3_pgens(CDR3_seq = tr.clone_df.cdr3_a_aa)

  tr.clone_df.to_csv(f"tcrdist/{re.sub('.csv', '', i)}_pgens.csv", index = False)

  tmp = pd.DataFrame(tr.pw_alpha)
  tmp.columns = tr.clone_df.id_nt.to_list()

  tmp.to_csv(f"tcrdist/{re.sub('.csv', '', i)}_tcrdistmatrix.csv", index = True, compression = 'gzip')

!tar -czvf tcrdist.tar.gz tcrdist
files.download("/content/tcrdist.tar.gz")

/usr/local/lib/python3.12/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/python3.12/dist-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")


tcrdist/
tcrdist/0_complete-data_tcrdistmatrix.csv
tcrdist/0_complete-data_pgens.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>